<h3>Token</h3>

In [ ]:
T_KEYWORD = "<keyword %s>"
T_OP = "<op %s>"
T_INT = "<int %s>"
T_STRING = "<string %s>"
T_IDENTIF = "<id %s>"
T_SPECIAL = "<special %s>"
T_PUNCT = "<punct %s>"
T_DOT = "<dot>"
T_CONDITIONAL_OP = "<conditional_op %s>"
T_COMMENT = "<comment>"

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def tokenize_line(line, line_number):
    tokens = []
    i = 0
    n = len(line)

    while i < n:
        if line[i].isspace():
            i += 1
        elif line[i] == '#':  # Comment
            tokens.append(T_COMMENT)
            break  # Skip the rest of the line
        elif line[i].isalpha():
            start = i
            while i < n and (line[i].isalnum() or line[i] == '_'):
                i += 1
            word = line[start:i]
            if word in ["var", "func", "if", "elif", "else", "return", "object", "init"]:
                tokens.append(T_KEYWORD % word)
            elif word in ["true", "false", "null", "end", "main"]:
                tokens.append(T_SPECIAL % word)
            else:
                tokens.append(T_IDENTIF % word)
        elif line[i].isdigit():
            start = i
            while i < n and line[i].isdigit():
                i += 1
            tokens.append(T_INT % line[start:i])
        elif line[i] == '"':
            start = i
            i += 1
            while i < n and line[i] != '"':
                i += 1
            if i >= n:
                print(f"Erro: String não fechada na linha {line_number}")
                raise StopExecution
            i += 1
            tokens.append(T_STRING % line[start:i])
        elif line[i] in "=<>!+-*/":
            start = i
            i += 1
            if i < n and line[i] == "=":
                i += 1
            tokens.append(T_OP % line[start:i])
        elif line[i] in "(),[]{}":
            tokens.append(T_PUNCT % line[i])
            i += 1
        elif line[i] in "?:":
            tokens.append(T_CONDITIONAL_OP % line[i])
            i += 1
        elif line[i] == '.':
            tokens.append(T_DOT)
            i += 1
        else:
            print(f"Erro: Caractere não reconhecido '{line[i]}' na linha {line_number}")
            raise StopExecution

    return tokens

def tokenize():

    try:
        token_total = [];
        with open('codigo.x', 'r') as f:
            lines = f.read().splitlines()

        line_number = 0
        for line in lines:
            line_number += 1
            tokens = tokenize_line(line, line_number)
            token_total.extend(tokens)
            #print(tokens)

        return token_total

    except StopExecution:
        print("Execução parada devido a erro.")

<h3>Analisador Lexico - AFD </h3>

In [ ]:
def analisador_lexico(filename):
    try:
        tokens_total = [];
        with open(filename, 'r') as f:
            lines = f.read().splitlines()

        line_number = 0
        for line in lines:
            line_number += 1
            tokens = tokenize_line(line, line_number)
            tokens_total.extend(tokens)
            #print(tokens)

        return tokens_total

    except StopExecution:
        print("Execução parada devido a erro.")

<h3>Analisador Sinatico - Parser </h3>

In [1]:
class Parser():
    def __init__(self, tokens):
        self.tokens = [Token(t.split(" ")[0], t.split(" ")[1] if len(t.split(" ")) > 1 else None) for t in tokens]
        self.pos = -1
        self.token_atual = None
        self.symbol_table = {}
        self.proximo()

    def proximo(self):
        self.pos += 1
        
        if self.pos >= len(self.tokens):
            self.token_atual = Token("EOF")
        else:    
            self.token_atual = self.tokens[self.pos]

        print(self.token_atual)
        return self.token_atual

    def erro(self):
        raise Exception('Erro de sintaxe. %s' % (self.token_atual))

    def use(self, tipo, valor=None):
        if self.token_atual.tipo != tipo:
            self.erro()
        elif valor is not None and self.token_atual.valor != valor:
            self.erro()
        else:
            self.proximo()

    def statement(self):
        if self.token_atual.tipo == "T_IDENTIF":
            self.assignment_statement()
        elif self.token_atual.tipo == "T_KEYWORD" and self.token_atual.valor in ["if", "elif", "else"]:
            self.conditional_statement()
        elif self.token_atual.tipo == "T_KEYWORD" and self.token_atual.valor in ["for", "while"]:
            self.loop_statement()
        elif self.token_atual.tipo == "T_KEYWORD" and self.token_atual.valor == "func":
            self.function_definition()
        # TODO: Adicione outros tipos de instruções conforme sua gramática

    def assignment_statement(self):
        var_name = self.token_atual.valor
        self.use("T_IDENTIF")
        self.use("T_OP", "=")
        value = self.expression()
        self.use("T_PUNCT", ";")
        self.symbol_table[var_name] = value

    def conditional_statement(self):
        if self.token_atual.valor == "if":
            self.use("T_KEYWORD", "if")
            self.use("T_PUNCT", "(")
            self.expression()  # Avalia a expressão do if, mas não faz nada com ela aqui. Normalmente, isso conduziria algum tipo de desvio condicional.
            self.use("T_PUNCT", ")")
            self.statement()
        elif self.token_atual.valor == "elif":
            self.use("T_KEYWORD", "elif")
            self.use("T_PUNCT", "(")
            self.expression()
            self.use("T_PUNCT", ")")
            self.statement()
        elif self.token_atual.valor == "else":
            self.use("T_KEYWORD", "else")
            self.statement()

    def loop_statement(self):
        if self.token_atual.valor == "for":
            self.use("T_KEYWORD", "for")
            self.use("T_PUNCT", "(")
            self.assignment_statement()  # Inicialização, ex: i = 0;
            self.expression()            # Condição, ex: i < 10;
            self.assignment_statement()  # Incremento, ex: i++;
            self.use("T_PUNCT", ")")
            self.statement()
        elif self.token_atual.valor == "while":
            self.use("T_KEYWORD", "while")
            self.use("T_PUNCT", "(")
            self.expression()            # Condição
            self.use("T_PUNCT", ")")
            self.statement()

    def function_definition(self):
        self.use("T_KEYWORD", "func")
        function_name = self.token_atual.valor
        self.use("T_IDENTIF")
        self.use("T_PUNCT", "(")
        # Para simplificar, estamos omitindo parâmetros de função aqui. Você deve expandir isso.
        self.use("T_PUNCT", ")")
        # Por enquanto, vamos assumir que toda função termina com um bloco.
        self.statement()

    def expression(self):
        # Aqui, mantemos o código para expressões como anteriormente, mas você pode querer expandi-lo.
        left = self.term()

        while self.token_atual.tipo == "T_OP" and self.token_atual.valor in ['+', '-']:
            if self.token_atual.valor == '+':
                self.use("T_OP", '+')
                left = left + self.term()
            else:
                self.use("T_OP", '-')
                left = left - self.term()

        return left

    # ... [outros métodos como term e factor permanecem inalterados] ...

    def start(self):
        while self.token_atual.tipo != "EOF":
            self.statement()


<h3>Compilador</h3>

In [ ]:
def main():
    filename = 'codigo.x'
    print(f"Lendo o arquivo {filename}...")

    # Tokenização
    print("\nTokenização:")
    tokens = tokenize(filename)
    
    # Análise Sintática (Parser)
    print("\nAnálise Sintática:")
    parser = Parser(tokens)
    try:
        parser.start()
        print("Análise sintática concluída com sucesso!")
    except Exception as e:
        print(f"Erro no parser: {e}")

    states = {
        'tokens': tokens,
        'symbol_table': parser.symbol_table
    }
    print("\nEstados salvos:", states)


if __name__ == '__main__':
    main()
